In [ ]:
#!/usr/bin/env python3
"""
Algorithm Performance Analysis Script
Analyzes runtimes and objective values across different algorithms and instance sizes
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style for better-looking plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Configuration
result_dir = "results"
req_sizes = ["50", "100", "200", "500"]
algs = ["deterministic", "random", "local_search", "vnd", "beam_search", "simulated_annealing"]

# Create plots directory
plots_dir = "plots"
os.makedirs(plots_dir, exist_ok=True)
print(f"Plots will be saved to: {plots_dir}")

# Load all data
print("\n" + "="*80)
print("LOADING DATA")
print("="*80)
all_data = []

for req_size in req_sizes:
    for alg in algs:
        file_path = os.path.join(result_dir, req_size, f"{alg}.csv")
        
        if os.path.exists(file_path):
            try:
                df = pd.read_csv(file_path)
                df['algorithm'] = alg
                df['instance_size'] = int(req_size)
                all_data.append(df)
                print(f"✓ Loaded {alg} for size {req_size}: {len(df)} instances")
            except Exception as e:
                print(f"✗ Error loading {file_path}: {e}")
        else:
            print(f"  Missing: {file_path}")

# Combine all data
if all_data:
    df_all = pd.concat(all_data, ignore_index=True)
    print(f"\nTotal records loaded: {len(df_all)}")
    print(f"Algorithms found: {df_all['algorithm'].unique()}")
    print(f"Instance sizes found: {sorted(df_all['instance_size'].unique())}")
else:
    print("No data loaded! Please check your file paths.")
    exit(1)

# Summary statistics
print("\n" + "="*80)
print("SUMMARY STATISTICS")
print("="*80)
summary_stats = df_all.groupby(['algorithm', 'instance_size']).agg({
    'time_seconds': ['mean', 'std', 'min', 'max'],
    'objective_value': ['mean', 'std', 'min', 'max'],
    'jain_fairness': ['mean', 'std', 'min', 'max'],
    'instance_name': 'count'
}).round(4)

summary_stats.columns = ['_'.join(col).strip() for col in summary_stats.columns.values]
summary_stats = summary_stats.rename(columns={'instance_name_count': 'num_instances'})
print(summary_stats)

# Save summary to CSV
summary_stats.to_csv(os.path.join(plots_dir, 'summary_statistics.csv'))
print(f"\nSummary statistics saved to {plots_dir}/summary_statistics.csv")

# Plot 1: Runtime Analysis
print("\n" + "="*80)
print("GENERATING PLOTS")
print("="*80)
print("1. Runtime Analysis...")
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1.1: Box plot of runtimes by algorithm
ax = axes[0, 0]
df_all.boxplot(column='time_seconds', by='algorithm', ax=ax)
ax.set_title('Runtime Distribution by Algorithm', fontsize=14, fontweight='bold')
ax.set_xlabel('Algorithm', fontsize=12)
ax.set_ylabel('Time (seconds)', fontsize=12)
plt.sca(ax)
plt.xticks(rotation=45, ha='right')

# Plot 1.2: Average runtime by instance size and algorithm
ax = axes[0, 1]
avg_runtime = df_all.groupby(['instance_size', 'algorithm'])['time_seconds'].mean().unstack()
avg_runtime.plot(ax=ax, marker='o', linewidth=2)
ax.set_title('Average Runtime vs Instance Size', fontsize=14, fontweight='bold')
ax.set_xlabel('Instance Size', fontsize=12)
ax.set_ylabel('Average Time (seconds)', fontsize=12)
ax.legend(title='Algorithm', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_yscale('log')

# Plot 1.3: Runtime heatmap
ax = axes[1, 0]
pivot_runtime = df_all.pivot_table(values='time_seconds', 
                                   index='algorithm', 
                                   columns='instance_size', 
                                   aggfunc='mean')
sns.heatmap(pivot_runtime, annot=True, fmt='.4f', cmap='YlOrRd', ax=ax, cbar_kws={'label': 'Time (seconds)'})
ax.set_title('Average Runtime Heatmap', fontsize=14, fontweight='bold')
ax.set_xlabel('Instance Size', fontsize=12)
ax.set_ylabel('Algorithm', fontsize=12)

# Plot 1.4: Violin plot for selected instance sizes
ax = axes[1, 1]
selected_sizes = [int(s) for s in req_sizes if int(s) in df_all['instance_size'].values][:4]
df_selected = df_all[df_all['instance_size'].isin(selected_sizes)]
if not df_selected.empty:
    sns.violinplot(data=df_selected, x='algorithm', y='time_seconds', ax=ax)
    ax.set_title(f'Runtime Distribution (Selected Sizes: {selected_sizes})', fontsize=14, fontweight='bold')
    ax.set_xlabel('Algorithm', fontsize=12)
    ax.set_ylabel('Time (seconds)', fontsize=12)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.savefig(os.path.join(plots_dir, 'runtime_analysis.png'), dpi=300, bbox_inches='tight')
print(f"   ✓ Saved: {plots_dir}/runtime_analysis.png")
plt.close()

# Plot 2: Objective Value Analysis
print("2. Objective Value Analysis...")
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 2.1: Box plot of objective values by algorithm
ax = axes[0, 0]
df_all.boxplot(column='objective_value', by='algorithm', ax=ax)
ax.set_title('Objective Value Distribution by Algorithm', fontsize=14, fontweight='bold')
ax.set_xlabel('Algorithm', fontsize=12)
ax.set_ylabel('Objective Value', fontsize=12)
plt.sca(ax)
plt.xticks(rotation=45, ha='right')

# Plot 2.2: Average objective value by instance size and algorithm
ax = axes[0, 1]
avg_objective = df_all.groupby(['instance_size', 'algorithm'])['objective_value'].mean().unstack()
avg_objective.plot(ax=ax, marker='o', linewidth=2)
ax.set_title('Average Objective Value vs Instance Size', fontsize=14, fontweight='bold')
ax.set_xlabel('Instance Size', fontsize=12)
ax.set_ylabel('Average Objective Value', fontsize=12)
ax.legend(title='Algorithm', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)

# Plot 2.3: Objective value heatmap
ax = axes[1, 0]
pivot_objective = df_all.pivot_table(values='objective_value', 
                                     index='algorithm', 
                                     columns='instance_size', 
                                     aggfunc='mean')
sns.heatmap(pivot_objective, annot=True, fmt='.1f', cmap='RdYlGn', ax=ax, 
            cbar_kws={'label': 'Objective Value'})
ax.set_title('Average Objective Value Heatmap', fontsize=14, fontweight='bold')
ax.set_xlabel('Instance Size', fontsize=12)
ax.set_ylabel('Algorithm', fontsize=12)

# Plot 2.4: Standard deviation of objective values
ax = axes[1, 1]
std_objective = df_all.groupby(['instance_size', 'algorithm'])['objective_value'].std().unstack()
std_objective.plot(ax=ax, marker='s', linewidth=2, linestyle='--')
ax.set_title('Objective Value Standard Deviation vs Instance Size', fontsize=14, fontweight='bold')
ax.set_xlabel('Instance Size', fontsize=12)
ax.set_ylabel('Standard Deviation', fontsize=12)
ax.legend(title='Algorithm', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(plots_dir, 'objective_value_analysis.png'), dpi=300, bbox_inches='tight')
print(f"   ✓ Saved: {plots_dir}/objective_value_analysis.png")
plt.close()

# Plot 3: Jain Fairness Analysis
print("3. Jain Fairness Analysis...")
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 3.1: Box plot of Jain fairness by algorithm
ax = axes[0, 0]
df_all.boxplot(column='jain_fairness', by='algorithm', ax=ax)
ax.set_title('Jain Fairness Distribution by Algorithm', fontsize=14, fontweight='bold')
ax.set_xlabel('Algorithm', fontsize=12)
ax.set_ylabel('Jain Fairness Index', fontsize=12)
ax.set_ylim([0.95, 1.005])
plt.sca(ax)
plt.xticks(rotation=45, ha='right')

# Plot 3.2: Average Jain fairness by instance size and algorithm
ax = axes[0, 1]
avg_fairness = df_all.groupby(['instance_size', 'algorithm'])['jain_fairness'].mean().unstack()
avg_fairness.plot(ax=ax, marker='o', linewidth=2)
ax.set_title('Average Jain Fairness vs Instance Size', fontsize=14, fontweight='bold')
ax.set_xlabel('Instance Size', fontsize=12)
ax.set_ylabel('Average Jain Fairness', fontsize=12)
ax.legend(title='Algorithm', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_ylim([0.95, 1.005])

# Plot 3.3: Fairness heatmap
ax = axes[1, 0]
pivot_fairness = df_all.pivot_table(values='jain_fairness', 
                                    index='algorithm', 
                                    columns='instance_size', 
                                    aggfunc='mean')
sns.heatmap(pivot_fairness, annot=True, fmt='.4f', cmap='YlGn', ax=ax, 
            cbar_kws={'label': 'Jain Fairness'}, vmin=0.95, vmax=1.0)
ax.set_title('Average Jain Fairness Heatmap', fontsize=14, fontweight='bold')
ax.set_xlabel('Instance Size', fontsize=12)
ax.set_ylabel('Algorithm', fontsize=12)

# Plot 3.4: Histogram of fairness values
ax = axes[1, 1]
for alg in df_all['algorithm'].unique():
    data = df_all[df_all['algorithm'] == alg]['jain_fairness']
    ax.hist(data, alpha=0.5, bins=30, label=alg, edgecolor='black')
ax.set_title('Jain Fairness Distribution Histogram', fontsize=14, fontweight='bold')
ax.set_xlabel('Jain Fairness Index', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.legend(title='Algorithm')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(os.path.join(plots_dir, 'jain_fairness_analysis.png'), dpi=300, bbox_inches='tight')
print(f"   ✓ Saved: {plots_dir}/jain_fairness_analysis.png")
plt.close()

# Plot 4: Performance Trade-offs
print("4. Performance Trade-offs...")
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 4.1: Runtime vs Objective Value scatter
ax = axes[0, 0]
for alg in df_all['algorithm'].unique():
    data = df_all[df_all['algorithm'] == alg]
    ax.scatter(data['time_seconds'], data['objective_value'], 
              alpha=0.6, s=50, label=alg, edgecolors='black', linewidth=0.5)
ax.set_title('Runtime vs Objective Value', fontsize=14, fontweight='bold')
ax.set_xlabel('Time (seconds)', fontsize=12)
ax.set_ylabel('Objective Value', fontsize=12)
ax.legend(title='Algorithm')
ax.grid(True, alpha=0.3)
ax.set_xscale('log')

# Plot 4.2: Runtime vs Jain Fairness scatter
ax = axes[0, 1]
for alg in df_all['algorithm'].unique():
    data = df_all[df_all['algorithm'] == alg]
    ax.scatter(data['time_seconds'], data['jain_fairness'], 
              alpha=0.6, s=50, label=alg, edgecolors='black', linewidth=0.5)
ax.set_title('Runtime vs Jain Fairness', fontsize=14, fontweight='bold')
ax.set_xlabel('Time (seconds)', fontsize=12)
ax.set_ylabel('Jain Fairness Index', fontsize=12)
ax.legend(title='Algorithm')
ax.grid(True, alpha=0.3)
ax.set_xscale('log')
ax.set_ylim([0.95, 1.005])

# Plot 4.3: Objective Value vs Jain Fairness scatter
ax = axes[1, 0]
for alg in df_all['algorithm'].unique():
    data = df_all[df_all['algorithm'] == alg]
    ax.scatter(data['objective_value'], data['jain_fairness'], 
              alpha=0.6, s=50, label=alg, edgecolors='black', linewidth=0.5)
ax.set_title('Objective Value vs Jain Fairness', fontsize=14, fontweight='bold')
ax.set_xlabel('Objective Value', fontsize=12)
ax.set_ylabel('Jain Fairness Index', fontsize=12)
ax.legend(title='Algorithm')
ax.grid(True, alpha=0.3)
ax.set_ylim([0.95, 1.005])

# Plot 4.4: Normalized performance comparison
ax = axes[1, 1]
available_sizes = sorted(df_all['instance_size'].unique())
if available_sizes:
    selected_size = available_sizes[0]
    df_radar = df_all[df_all['instance_size'] == selected_size].groupby('algorithm').agg({
        'time_seconds': 'mean',
        'objective_value': 'mean',
        'jain_fairness': 'mean'
    })
    
    # Normalize metrics (higher is better)
    df_radar['time_norm'] = 1 - (df_radar['time_seconds'] - df_radar['time_seconds'].min()) / (df_radar['time_seconds'].max() - df_radar['time_seconds'].min() + 1e-10)
    df_radar['obj_norm'] = (df_radar['objective_value'] - df_radar['objective_value'].min()) / (df_radar['objective_value'].max() - df_radar['objective_value'].min() + 1e-10)
    df_radar['fair_norm'] = df_radar['jain_fairness']
    
    metrics = ['Speed\\n(norm)', 'Objective\\n(norm)', 'Fairness\\n(norm)']
    x_pos = np.arange(len(metrics))
    width = 0.15
    
    for i, alg in enumerate(df_radar.index):
        values = [df_radar.loc[alg, 'time_norm'], 
                 df_radar.loc[alg, 'obj_norm'], 
                 df_radar.loc[alg, 'fair_norm']]
        ax.bar(x_pos + i*width, values, width, label=alg, alpha=0.8)
    
    ax.set_title(f'Normalized Performance Comparison (Size {selected_size})', fontsize=14, fontweight='bold')
    ax.set_ylabel('Normalized Score (0-1)', fontsize=12)
    ax.set_xticks(x_pos + width * (len(df_radar) - 1) / 2)
    ax.set_xticklabels(metrics, fontsize=10)
    ax.legend(title='Algorithm', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.set_ylim([0, 1.1])
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(os.path.join(plots_dir, 'performance_tradeoffs.png'), dpi=300, bbox_inches='tight')
print(f"   ✓ Saved: {plots_dir}/performance_tradeoffs.png")
plt.close()

# Plot 5: Scalability Analysis
print("5. Scalability Analysis...")
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 5.1: Log-log plot of runtime scaling
ax = axes[0]
avg_runtime_by_size = df_all.groupby(['instance_size', 'algorithm'])['time_seconds'].mean().unstack()
for alg in avg_runtime_by_size.columns:
    data = avg_runtime_by_size[alg].dropna()
    ax.plot(data.index, data.values, marker='o', linewidth=2, label=alg, markersize=8)
ax.set_title('Runtime Scaling (Log-Log)', fontsize=14, fontweight='bold')
ax.set_xlabel('Instance Size (log scale)', fontsize=12)
ax.set_ylabel('Average Time in seconds (log scale)', fontsize=12)
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend(title='Algorithm')
ax.grid(True, alpha=0.3, which='both')

# Plot 5.2: Relative performance compared to fastest algorithm
ax = axes[1]
min_times = df_all.groupby('instance_size')['time_seconds'].min()
relative_performance = {}

for alg in df_all['algorithm'].unique():
    alg_times = df_all[df_all['algorithm'] == alg].groupby('instance_size')['time_seconds'].mean()
    relative_performance[alg] = (alg_times / min_times).dropna()

for alg, rel_perf in relative_performance.items():
    ax.plot(rel_perf.index, rel_perf.values, marker='s', linewidth=2, label=alg, markersize=8)

ax.set_title('Relative Performance vs Best Algorithm', fontsize=14, fontweight='bold')
ax.set_xlabel('Instance Size', fontsize=12)
ax.set_ylabel('Slowdown Factor (vs fastest)', fontsize=12)
ax.axhline(y=1, color='red', linestyle='--', alpha=0.5, label='Fastest')
ax.legend(title='Algorithm')
ax.grid(True, alpha=0.3)
ax.set_yscale('log')

plt.tight_layout()
plt.savefig(os.path.join(plots_dir, 'scalability_analysis.png'), dpi=300, bbox_inches='tight')
print(f"   ✓ Saved: {plots_dir}/scalability_analysis.png")
plt.close()

# Statistical Comparison
print("\n" + "="*80)
print("STATISTICAL COMPARISON")
print("="*80)

print("\n1. Average Runtime Ranking (lower is better):")
runtime_ranking = df_all.groupby('algorithm')['time_seconds'].mean().sort_values()
for i, (alg, time) in enumerate(runtime_ranking.items(), 1):
    print(f"   {i}. {alg:20s}: {time:.6f} seconds")

print("\n2. Average Objective Value Ranking (higher is better):")
objective_ranking = df_all.groupby('algorithm')['objective_value'].mean().sort_values(ascending=False)
for i, (alg, obj) in enumerate(objective_ranking.items(), 1):
    print(f"   {i}. {alg:20s}: {obj:.2f}")

print("\n3. Average Jain Fairness Ranking (higher is better):")
fairness_ranking = df_all.groupby('algorithm')['jain_fairness'].mean().sort_values(ascending=False)
for i, (alg, fair) in enumerate(fairness_ranking.items(), 1):
    print(f"   {i}. {alg:20s}: {fair:.6f}")

print("\n4. Instance Count by Size:")
size_counts = df_all.groupby('instance_size')['instance_name'].count().sort_index()
for size, count in size_counts.items():
    print(f"   Size {size:6d}: {count:4d} instances")

# Export results
print("\n" + "="*80)
print("EXPORTING RESULTS")
print("="*80)

output_file = os.path.join(plots_dir, 'complete_results.csv')
df_all.to_csv(output_file, index=False)
print(f"✓ Complete results exported to: {output_file}")

comparison_file = os.path.join(plots_dir, 'algorithm_comparison_by_size.csv')
comparison = df_all.groupby(['instance_size', 'algorithm']).agg({
    'time_seconds': ['mean', 'std'],
    'objective_value': ['mean', 'std'],
    'jain_fairness': ['mean', 'std'],
    'instance_name': 'count'
}).round(6)
comparison.to_csv(comparison_file)
print(f"✓ Algorithm comparison exported to: {comparison_file}")

print("\n" + "="*80)
print("ANALYSIS COMPLETE")
print("="*80)
print(f"\nAll plots and results saved to: {plots_dir}/")
print("  - runtime_analysis.png")
print("  - objective_value_analysis.png")
print("  - jain_fairness_analysis.png")
print("  - performance_tradeoffs.png")
print("  - scalability_analysis.png")
print("  - summary_statistics.csv")
print("  - complete_results.csv")
print("  - algorithm_comparison_by_size.csv")

Plots will be saved to: plots

LOADING DATA
✓ Loaded deterministic for size 50: 30 instances
✓ Loaded random for size 50: 30 instances
  Missing: results/50/local_search.csv
  Missing: results/50/vnd.csv
✓ Loaded beam_search for size 50: 30 instances
✓ Loaded simulated_annealing for size 50: 30 instances
✓ Loaded deterministic for size 100: 30 instances
✓ Loaded random for size 100: 30 instances
  Missing: results/100/local_search.csv
  Missing: results/100/vnd.csv
  Missing: results/100/beam_search.csv
✓ Loaded simulated_annealing for size 100: 30 instances
✓ Loaded deterministic for size 200: 30 instances
✓ Loaded random for size 200: 30 instances
  Missing: results/200/local_search.csv
  Missing: results/200/vnd.csv
  Missing: results/200/beam_search.csv
  Missing: results/200/simulated_annealing.csv
✓ Loaded deterministic for size 500: 30 instances
✓ Loaded random for size 500: 30 instances
  Missing: results/500/local_search.csv
  Missing: results/500/vnd.csv
  Missing: results/50